In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data
import xarray as xr
import h5py

# Helpful
import time
import datetime
import itertools
from itertools import product

# My Methods
import importlib
import CRPS
import EMOS
from CRPS import *
from EMOS import *

2023-05-15 12:36:28.504290: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-05-15 12:36:28.504326: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

In [13]:
importlib.reload(CRPS)
importlib.reload(EMOS)

<module 'EMOS' from '/home/dchen/BA_CH_EN/EMOS.py'>

<IPython.core.display.Javascript object>

### Goal of this notebook: Implement EMOS and train it on train dataset

#### 1. Load dataset

In [9]:
t2m_train = xr.open_dataset("/Data/Delong_BA_Data/Mean_ens_std/t2m_train.h5")
t2m_X_train = t2m_train.t2m_train
t2m_y_train = t2m_train.t2m_truth

<IPython.core.display.Javascript object>

#### 2. Prepare Data
Make datasets for all 32 lead times

In [164]:
t2m_X_train_glob_mean = []
t2m_X_train_glob_std = []
t2m_y_train_glob_truth = []
for i in range(t2m_train.phony_dim_1.shape[0]):
    t2m_X_train_glob_mean.append(
        t2m_X_train.isel(phony_dim_4=0, phony_dim_1=i).values.flatten()
    )
    t2m_X_train_glob_std.append(
        t2m_X_train.isel(phony_dim_4=1, phony_dim_1=i).values.flatten()
    )
    t2m_y_train_glob_truth.append(t2m_y_train.isel(phony_dim_1=i).values.flatten())

<IPython.core.display.Javascript object>

In [165]:
# Only necessary for Baseline, not for EMOS
# Change all zeros in std to a small value epsilon
epsilon = 1e-9  # Small epsilon to add to zeros in std
for i in range(len(t2m_X_train_glob_std)):
    t2m_X_train_glob_std[i][(t2m_X_train_glob_std[i] == 0)] += epsilon

<IPython.core.display.Javascript object>

In [177]:
# Decide on which lead_time to test
lead_time = 1

<IPython.core.display.Javascript object>

#### 3. Baseline

In [178]:
crps_baseline = crps_normal(
    mu=t2m_X_train_glob_mean[lead_time],
    sigma=t2m_X_train_glob_std[lead_time],
    y=t2m_y_train_glob_truth[lead_time],
)

<IPython.core.display.Javascript object>

In [179]:
crps_baseline.mean()

0.10622623690424136

<IPython.core.display.Javascript object>

#### 4. Train global Emos

In [180]:
EMOS_glob = build_EMOS_network_keras(compile=True)

<IPython.core.display.Javascript object>

In [181]:
EMOS_glob.fit(
    [t2m_X_train_glob_mean[lead_time], t2m_X_train_glob_std[lead_time]],
    t2m_y_train_glob_truth[lead_time],
    batch_size=5000,
    epochs=5,
    validation_split=0.2,
)

Epoch 1/5
3567/3567 [==============================] - 10s 2ms/step - loss: 0.0987 - val_loss: 0.0991
Epoch 2/5
3567/3567 [==============================] - 9s 2ms/step - loss: 0.0983 - val_loss: 0.0991
Epoch 3/5
3567/3567 [==============================] - 9s 2ms/step - loss: 0.0983 - val_loss: 0.0991
Epoch 4/5
3567/3567 [==============================] - 9s 2ms/step - loss: 0.0983 - val_loss: 0.0991
Epoch 5/5
3567/3567 [==============================] - 9s 2ms/step - loss: 0.0983 - val_loss: 0.0991


<IPython.core.display.Javascript object>